<a href="https://colab.research.google.com/github/ucfilho/Raianars_paper_one_revisited/blob/main/After_asked_reviews_one_2022/LSHADE/CFRANNK_1_3_LSHADE_1_3_DE_LPSR_F15_180_30cec2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
############################### DADOS DE ENTRADA 
 
# this code only run from F22 to F25 (you just need to change this line the pararameters and type =cool= 
#          to go to line to change IFrank, de best or your combination
RUNS=50 #NÚMERO DE RODADAS
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
ITE=1000 #10 # PONTOS quantas vezes vai alteral NPAR 
NPAR=100
mut=0.5
crossp=0.5
Fi=mut;CRi=crossp;# para impressão
fator = 1.0/3.0; 
#fator = 1.0 # just LSHADE...
mut2=0.5;crossp2=0.5;mut3=0.5;crossp3=0.5;mut4=0.25;crossp4=0.25;
mut_best  =0.5 ; crossp_best =0.5
maxNP = 180
minNP = 30
maxFES =300000
FIX = 100 # Number of adjusts random walk
H = 6
kH = 0 # kH is used to work with H  
terminal = 0.1
p=0.1
SF=[];SCR=[];MF=[];MCR=[];Xarq=[]


In [2]:
############################### Importar códigos
!git clone https://github.com/ucfilho/optim_raianars_Insight/ #clonar do Github
%cd optim_raianars_Insight
import de_soma_Insight
import de_bounds_matriz
import Go2Ann
import randomWALK
import annDE_FES_new
import classicDE_FES_new
import stepsWALK_new
import LShade_new
from pandas import read_csv

fatal: destination path 'optim_raianars_Insight' already exists and is not an empty directory.
/content/optim_raianars_Insight


In [3]:
!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
%cd Raianars_paper_One

fatal: destination path 'Raianars_paper_One' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One


In [4]:
#import Go2Ann
#import de_soma_Insight
#import de_bounds_matriz
#import Function
import Go2MinMax
from AvaliaX import AvaliaX
import numpy as np
import scipy.stats
import random
from GeraIndices import GeraIndices
import numpy as np
import time
import matplotlib.pyplot as plt
from random import randint
from scipy import optimize
import pandas as pd
from sklearn.metrics import confusion_matrix

In [5]:
############################### Importar códigos
!git clone https://github.com/ucfilho/optim_raianars_Insight/ #clonar do Github
%cd optim_raianars_Insight
import de_soma_Insight
import de_bounds_matriz

fatal: destination path 'optim_raianars_Insight' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight


In [6]:
#!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
#%cd Raianars_paper_One

!git clone https://github.com/ucfilho/Raianars_Routines #clonar do Github
%cd Raianars_Routines

fatal: destination path 'Raianars_Routines' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines


In [7]:
!git clone https://github.com/ucfilho/functions_cec2014_data
%cd functions_cec2014_data
import F1_to_F27_cec2014_zero_to_hero
import cec2014_names
import cec2014_shift_rotate

fatal: destination path 'functions_cec2014_data' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data


In [8]:
function_select = 'F15'
Function = F1_to_F27_cec2014_zero_to_hero.F15
Shift_Rotate= cec2014_shift_rotate.F15
problem_size = 30 # possibles 2, 10, 20, 30, 50, 100

In [9]:
dim = problem_size 

In [10]:
f_matrix = "M_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
shift_data_file="shift_data_"+function_select[1:]+".txt"
F_bias = cec2014_names.BiasValue()[function_select]
type_fun = cec2014_names.TypeFunctions()[function_select]
Function_CEC2014 =function_select + "_" + cec2014_names.NamesFunctions()[function_select]+"_D_"+str(problem_size)

In [11]:
!pip install --upgrade --no-cache-dir gdown

In [12]:
# !gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
# https://drive.google.com/file/d/1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz/view?usp=sharing
!gdown --id 1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1yM-cGzp7F35bXZ-f80uoJHzGqAb52CPz
To: /content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/cec2014.zip
100% 36.1M/36.1M [00:00<00:00, 291MB/s]


In [13]:
import zipfile
file_name = zipfile.ZipFile('cec2014.zip', 'r')
file_name.extractall()

In [14]:
def load_shift_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values.reshape((-1))

def load_matrix_data__(data_file=None):
    data = read_csv(data_file, delimiter='\s+', index_col=False, header=None)
    return data.values

In [15]:
shift_data_a = load_shift_data__(shift_data_file)[:problem_size]
matrix_a = load_matrix_data__(f_matrix)
if (type_fun == 'hybrid'):
  f_shuffle = "shuffle_data_"+function_select[1:]+"_D"+str(problem_size) + ".txt"
  shuffle = (load_shift_data__(f_shuffle)[:problem_size] - np.ones(problem_size)).astype(int)
else:
  shuffle = matrix_a

In [16]:
SUPPORT_DIMENSION = [2, 10, 20, 30, 50, 100]
SUPPORT_DIMENSION_2 = [10, 20, 30, 50, 100]

#def Fun(solution,shift_data=shift_data_a, matrix=matrix_a, shuffle):#, bias=F_bias):
def CEC2014(solution, shift_data ,  matrix, shuffle):#, bias=F_bias):
  if(type_fun == 'shift_rotate'):
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = Function(z) + F_bias
  elif(type_fun =='hybrid'): # F17 to F22
    z = Shift_Rotate(solution , shift_data , matrix,shuffle)
    result = z + F_bias
 
  return result

In [17]:
 shift_data = shift_data_a
 matrix = matrix_a
 shuffle = shuffle
 def Hybrid(solution):
   resp = CEC2014(solution, shift_data = shift_data_a, matrix = matrix_a, shuffle = shuffle)
   return resp

In [18]:
!git clone https://github.com/ucfilho/Raianars_paper_One #clonar do Github
%cd Raianars_paper_One

fatal: destination path 'Raianars_paper_One' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/Raianars_paper_One


In [19]:
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)

bounds=[]
for k in range(len(MAX)):
  bounds.append((MIN[k],MAX[k]))

In [20]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

fatal: destination path 'ANN_read_matrix' already exists and is not an empty directory.
/content/optim_raianars_Insight/Raianars_paper_One/optim_raianars_Insight/Raianars_Routines/functions_cec2014_data/Raianars_paper_One/ANN_read_matrix


In [21]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')

In [22]:
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')

In [23]:
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)

In [24]:
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)

In [25]:
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')

In [26]:
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

In [27]:
##################################### 

TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

#dimensions=len(MAX)
dimensions=dim
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO


PAR=dim
Fun = Function
fobj=Fun

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
NPAR1=int(fator*NPAR);
NPAR2=int(fator*NPAR)
#NPAR2=int((NPAR-NPAR1)/3);
#NPAR3=int((NPAR-NPAR1)/3);
#NPAR4=int((NPAR-NPAR1)/3);
NPAR3 = NPAR-NPAR1-NPAR1
print('frac=',fator);
print('Npar1=',NPAR1);
print('Npar2=',NPAR2);
print('Npar3=',NPAR3);
#print('Npar4=',NPAR4);
print('Npar=',NPAR)

cols=dim

X1=np.zeros((NPAR1,cols)) # X1 is Frannk population
for i in range(NPAR1):
  for j in range(cols):
    r=np.random.random()
    X1[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
X2=np.zeros((NPAR2,cols)) # X2 is population DE (default DE)
for i in range(NPAR2):
  for j in range(cols):
    r=np.random.random()
    X2[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

X3=np.zeros((NPAR3,cols)) # X1 is Frannk population
for i in range(NPAR3):
  for j in range(cols):
    r=np.random.random()
    X3[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
'''    
X4=np.zeros((NPAR4,cols)) # X2 is population DE (default DE)
for i in range(NPAR4):
  for j in range(cols):
    r=np.random.random()
    X4[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
 ############################################################
  #bounds=[(5*dim, 15*dim)] * PONTOS # ITE
  #bounds=[(3*dim, 12*dim)] * PONTOS # ITE
  #bounds=[(4*dim, 16*dim)] * PONTOS
  
  # nao pode ser assim pq o bounds vale p o DE nao dos parametros
  #p=int(TOTAL/its/ITE) 
  #bounds=[(p, p)] * PONTOS
'''

frac= 0.3333333333333333
Npar1= 33
Npar2= 33
Npar3= 34
Npar= 100


'    \nX4=np.zeros((NPAR4,cols)) # X2 is population DE (default DE)\nfor i in range(NPAR4):\n  for j in range(cols):\n    r=np.random.random()\n    X4[i,j]=r*(MAX[j]-MIN[j])+MIN[j]\n    \n ############################################################\n  #bounds=[(5*dim, 15*dim)] * PONTOS # ITE\n  #bounds=[(3*dim, 12*dim)] * PONTOS # ITE\n  #bounds=[(4*dim, 16*dim)] * PONTOS\n  \n  # nao pode ser assim pq o bounds vale p o DE nao dos parametros\n  #p=int(TOTAL/its/ITE) \n  #bounds=[(p, p)] * PONTOS\n'

In [28]:
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for iii in range(RUNS):
    FES = 0 ; gen = 0 
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)

    NPAR = maxNP
    # fator = 1.0
    NPAR1=int(fator*NPAR);NPAR2=int(fator*NPAR);NPAR3 = NPAR-NPAR1-NPAR1
    '''
    NPAR2=int((NPAR-NPAR1)/3);
    NPAR3=int((NPAR-NPAR1)/3);
    NPAR4=int((NPAR-NPAR1)/3);
    NPAR1 = NPAR-3*NPAR2
    '''

    X1=np.zeros((NPAR1,cols)) # X1 is Frannk population
    for i in range(NPAR1):
      for j in range(cols):
        r=np.random.random()
        X1[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
       
    X2=np.zeros((NPAR2,cols)) # X2 is population DE (default DE)
    for i in range(NPAR2):
      for j in range(cols):
        r=np.random.random()
        X2[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    
    X3=np.zeros((NPAR3,cols)) # X2 is population DE (default DE)
    for i in range(NPAR3):
      for j in range(cols):
        r=np.random.random()
        X3[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    ''' 
    X4=np.zeros((NPAR4,cols)) # X2 is population DE (default DE)
    for i in range(NPAR4):
      for j in range(cols):
        r=np.random.random()
        X4[i,j]=r*(MAX[j]-MIN[j])+MIN[j]
    '''
    fit1 = np.asarray([fobj(ind) for ind in X1])
    best_idx = np.argmin(fit1)
    best1 = X1[best_idx]
    fbest1 = fit1[best_idx] 

    
    fit2 = np.asarray([fobj(ind) for ind in X2])
    best_idx = np.argmin(fit2)
    best2 = X1[best_idx]
    fbest2 = fit2[best_idx] 

    fit3 = np.asarray([fobj(ind) for ind in X3])
    best_idx = np.argmin(fit3)
    best3 = X3[best_idx]
    fbest3 = fit3[best_idx] 
    '''
    fit3 = np.asarray([fobj(ind) for ind in X3])
    fit4 = np.asarray([fobj(ind) for ind in X4])
    FES = FES + NPAR1 + NPAR2 + NPAR3+ NPAR4
    '''
    FES = FES + NPAR1 + NPAR2 + NPAR3
    fit_best =[]
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    #Fitness = np.asarray([Fun(ind, shift_data , matrix, shuffle) for ind in X1])
    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X1,fit1 )
    nrow,ncol=X1.shape
    FES = FES + nrow

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo

    Fuma = Fun  # Fuma era Fun
  
    MAT_INDo=GeraIndices(X1,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fuma)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo

    # Just to intialize variables:
    FESr = 0
    FOBEST1 = fbest1;FOBEST2 = fbest2;FOBEST3 = fbest3;FOBEST4 = 1E99
    #BEST1,BEST2,BEST3,BEST4  =  np.copy(X1[0,:]),np.copy(X2[0,:]),np.copy(X3[0,:]), np.copy(X4[0,:]) 
    BEST1  =  best1
    BEST2  =  best2
    BEST3  =  best3
    popsize = NPAR
    FUNCTION = []
    for i in range(H):
      MCR.append(0.5);MF.append(0.5) # line 3
    Narquive = popsize
    setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]
    while(FESr == 0 ):

      #print('FES',FES)

      gen = gen +1
      ##=cool=#####################################################################################
      '''
      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit2),np.copy(X2),BEST2,FOBEST2,FES
      fit2,X2,BEST2,FOBEST2,XY2,BEST_XY2,FES=classicDE_FES_new.de(bounds, mut2, crossp2,NPAR2, its,fobj,fit2,X2,FES) # de_bounds_matriz
      if(FES > maxFES):
        FESr = FES 
        fit2,X2,BEST2,FOBEST2,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
      
      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit3),np.copy(X3),BEST3,FOBEST3,FES
      fit3,X3,BEST3,FOBEST3,XY3,BEST_XY3,FES=classicDE_FES_new.de(bounds, mut3, crossp3,NPAR3, its,fobj,fit3,X3,FES) #de_bounds_matriz
      if(FES > maxFES): 
        FESr = FES
        fit3,X3,BEST3,FOBEST3,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback

      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit4),np.copy(X4),BEST4,FOBEST4,FES
      fit4,X4,BEST4,FOBEST4,XY4,BEST_XY4,FES=classicDE_FES_new.de(bounds, mut4, crossp4,NPAR4, its,fobj,fit4,X4,FES) # de_bounds_matriz
      if(FES > maxFES): 
        FESr = FES
        fit4,X4,BEST4,FOBEST4,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
      '''
      
      

      

      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit1),np.copy(X1),BEST1,FOBEST1,FES
      fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES = annDE_FES_new.de(MAX,MIN, mut, crossp, NPAR1, its,fobj,fit1,X1,FES)
      if(FES > maxFES):
        FESr = FES 
        fit1,X1,BEST1,FOBEST1,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback

      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit2),np.copy(X2),BEST2,FOBEST2,FES
      Xarqback = Xarq
      fit2,X2,BEST2,FOBEST2,XY,BEST_XY,FES,Xarq,setTUNE= LShade_new.LShade(MAX,MIN, NPAR2,fobj,setTUNE,best2,fbest2,fit2,X2,Xarq,FES)
      if(FES > maxFES):
        FESr = FES 
        fit2,X2,BEST2,FOBEST2,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback
        Xarq = np.copy(Xarqback)

      fback,Xback,BESTback,FOBESTback,FESback= np.copy(fit3),np.copy(X3),BEST3,FOBEST3,FES
      fit3,X3,BEST3,FOBEST3,XY3,BEST_XY3,FES=classicDE_FES_new.de(bounds, mut3, crossp3,NPAR3, its,fobj,fit3,X3,FES) #de_bounds_matriz
      if(FES > maxFES): 
        FESr = FES
        fit3,X3,BEST3,FOBEST3,FES = np.copy(fback),np.copy(Xback),BESTback,FOBESTback,FESback


      calc = (minNP-maxNP)/maxFES*FES+maxNP # Linear Population Size Reduction (LPSR)
      calc = int(np.round(calc)) # Linear Population Size Reduction (LPSR) 
      gen = gen + 1
      #fator = 1.0
      NPAR = calc
      NPAR1=int(fator*NPAR);
      NPAR2=int(fator*NPAR);
      NPAR3=NPAR-NPAR1-NPAR2
      #NPAR3=int((NPAR-NPAR1)/3);
      #NPAR4=int((NPAR-NPAR1)/3);
      #NPAR1 = NPAR-3*NPAR2

      '''
      if(NPAR2 > len(X2[:,0])):
        NPAR2 = len(X2[:,0])
      '''
      if(NPAR1 > len(X1[:,0])):
        NPAR1 = len(X1[:,0])
      if(NPAR2 > len(X2[:,0])):
        NPAR2 = len(X2[:,0])
      if(NPAR3 > len(X3[:,0])):
        NPAR3 = len(X3[:,0])    
          
      X1 = X1[:NPAR1,:];X2 = X2[:NPAR2,:];X3 = X3[:NPAR3,:];
      if(len(Xarq)>0) : Xarq = Xarq[:NPAR2,:]
       #X4 = X4[:NPAR4,:];
      fit1 = fit1[:NPAR1];fit2 = fit2[:NPAR2]; fit3 = fit3[:NPAR3]; # fit4 = fit4[:NPAR4];
      popsize = NPAR2
      Narquive = popsize
      NP = popsize

      X1[0,:] = BEST1;X1[1,:] = BEST2;X1[2,:] = BEST3;# X1[2,:] = BEST4;
      X2[0,:] = BEST1;X2[1,:] = BEST2;X2[2,:] = BEST3;#X2[3,:] = BEST2;
      X3[0,:] = BEST1;X3[1,:] = BEST2;X3[2,:] = BEST3;#X3[3,:] = BEST3;
      #X4[0,:] = BEST1;X4[1,:] = BEST2;X4[2,:] = BEST3;X4[3,:] = BEST4;
      fit1[0] = FOBEST1; fit1[1] = FOBEST2;fit1[2] = FOBEST3;
      fit2[0] = FOBEST1; fit2[1] = FOBEST2;fit2[2] = FOBEST3;
      fit3[0] = FOBEST1; fit3[1] = FOBEST2;fit3[2] = FOBEST3;
      '''
      X1[0,:] = BEST2;X1[1,:] = BEST3; X1[2,:] = BEST4;X1[3,:] = BEST1;
      X2[0,:] = BEST1;X2[1,:] = BEST3;X2[2,:] = BEST4;X2[3,:] = BEST2;
      X3[0,:] = BEST1;X3[1,:] = BEST2;X3[2,:] = BEST4;X3[3,:] = BEST3;
      X4[0,:] = BEST1;X4[1,:] = BEST2;X4[2,:] = BEST3;X4[3,:] = BEST4;
      fit1[0] = FOBEST1; fit2[1] = FOBEST2;fit3[0] = FOBEST3; fit4[1] = FOBEST4;
      '''
    '''
    for i in range(FIX):
      
        Fitback,Xback,BESTback,FOBESTback,FESback= np.copy(fit2), np.copy(X2),BEST2,FOBEST2,FES
        
        if ( FESr == 0):
          Fitback,Xback,BESTback,FOBESTback,FESback= np.copy(X2),BEST2,FOBEST2,FES
          fit2,X2,BEST2,FOBEST2,XY2,BEST_XY2,FESr=classicDE_FES_new.de(bounds,mut_best, crossp_best,NPAR2, its,fobj,fit2,X2,FES) # de_bounds_matriz
        if ( FESr > 0):
          fit2, X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback


        if(FES > maxFES): 
          fit2, X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback
        
        
        fit2,X2,BEST2,FOBEST2,XY,BEST_XY, FES= stepsWALK_new.fixWALK(Fun,BEST2,FOBEST2,NPAR2,tunePAR,MAX,MIN,fit2,X2,FIX)
        # print('FES',FES)

      ############################################################################################
      ############################################################################################
    
    if(FES > maxFES):
      fit2,X2,BEST2,FOBEST2,FES = np.copy(Fitback),np.copy(Xback),BESTback,FOBESTback,FESback

    FUNCTION.append(FOBEST2)
    '''
    #setTUNE= [SF,SCR,MF,MCR,p, terminal, Narquive]
    setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]
    print('run=',iii,'gen=',gen,'time=',(time.time()-start_time)/60, 'eval FES=', FES,'FO=',FOBEST1)  
    BEST_AGENTS.append(BEST_XY)

run= 0 gen= 7202 time= 2.0404401938120524 eval FES= 299998 FO= 2.2862503083383174
run= 1 gen= 7202 time= 3.964840336640676 eval FES= 299998 FO= 5.77796040590873
run= 2 gen= 7202 time= 5.808367602030436 eval FES= 299998 FO= 6.544236258076133
run= 3 gen= 7202 time= 7.651643558343252 eval FES= 299998 FO= 9.020384610385362
run= 4 gen= 7202 time= 9.492662171522776 eval FES= 299998 FO= 3.311555291747896
run= 5 gen= 7202 time= 11.338929375012716 eval FES= 299998 FO= 3.1450219128851526
run= 6 gen= 7202 time= 13.178439903259278 eval FES= 299998 FO= 8.066397439371181
run= 7 gen= 7202 time= 15.019404884179433 eval FES= 299998 FO= 4.866475262555944
run= 8 gen= 7202 time= 16.86677403450012 eval FES= 299998 FO= 4.6063081141267554
run= 9 gen= 7202 time= 18.72153252363205 eval FES= 299998 FO= 2.3043351111465435
run= 10 gen= 7202 time= 20.571446053187053 eval FES= 299998 FO= 6.02236028121476
run= 11 gen= 7202 time= 22.41155047416687 eval FES= 299998 FO= 4.72359942410626
run= 12 gen= 7202 time= 24.24741

In [29]:
NPAR1,NPAR2,NPAR3,X1.shape,X2.shape,X3.shape

(10, 10, 10, (10, 30), (10, 30), (10, 30))

In [30]:
FES,setTUNE,SCR,SF

(299998,
 [[],
  [],
  [0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   0.5,
   

In [31]:
#len(X2[1,:])
print(cols)

30


In [32]:
BEST_AGENTS=np.array(BEST_AGENTS)

In [33]:
NOMES=['DIVERSIDADE (DI)',  'DI RELATIVO','FRAC Its',
           'VELOC FOBJ','VELOC X','NP','Fo','CRo','fmin','fmax',
           'AD_fmin','AD_fmax','DELTA FOBJ','DELTA VX','r_fitness',
           'F','CR']
# BEST_INDEX= BEST_AGENTS[BEST_AGENTS[:,-1].argmin()]  
BEST_INDEX = np.argmin(BEST_AGENTS[:,-1])
#df=pd.DataFrame(MAT_stat[BEST_INDEX], columns=NOMES)



In [34]:
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()    

In [35]:
nome='SUB'+'_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('MCS-FRANNK','Fator F/DE= %.2f' %(fator),'NPAR=',NPAR,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
#print('BEST: ',BEST)
#print('MÉDIA: ',MEAN)
print('MÉDIA: ',MEAN[-1],'STD __:',STD)
print('MIN __:',MINFO, 'MAX__: ',MAXFO)
print(MEAN[-1])
print(STD)
print(MINFO)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)
# print('melhor=',BEST_AGENTS[BEST_INDEX])

MCS-FRANNK Fator F/DE= 0.33 NPAR= 30 Iterações= 1000 Dim= 30 Min/Max= -100 / 100 _ F15
SUB_Fo-0.5_CRo-0.5_D-30_Interv-1000 

MÉDIA:  4.881924934854844 STD __: 1.7900917180285643
MIN __: 2.2851572377021254 MAX__:  10.578731702874805
4.881924934854844
1.7900917180285643
2.2851572377021254
Tempo Médio= 111.578 segundos ---
50 Rodadas

BEST:  [ 0.76052096  0.60897161  0.72444431  0.17280348 -0.3953332  -0.17007002
 -0.30545422  0.63872812  0.84154761  0.71948024  0.95118029  0.90035182
  0.78758085  0.37013264 -0.21198252  0.46226338  0.22387228  0.03882881
 -0.55202399  0.84323772  0.73072296  0.5417595   0.04888226  0.2358013
 -0.18043579 -0.51576123  0.86122502  0.73205285  0.78554623  0.638495
  2.28515724]


In [36]:
MF,MCR,SF,SCR

([0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,

In [37]:
FES,Xarq

(299998, array([], dtype=float64))